In [1]:
import pandas as pd
import numpy as np

# 读取txt文件为DataFrame
file_path = '../cornell/formatted_movie_lines.txt'
data = pd.read_csv(file_path, sep='\t', header=None, names=['speaker1', 'speaker2'])

# 可选：查看前几行数据
print(data.head())

                                            speaker1  \
0  Can we make this quick?  Roxanne Korrine and A...   
1  Well, I thought we'd start with pronunciation,...   
2  Not the hacking and gagging and spitting part....   
3  You're asking me out.  That's so cute. What's ...   
4  No, no, it's my fault -- we didn't have a prop...   

                                            speaker2  
0  Well, I thought we'd start with pronunciation,...  
1  Not the hacking and gagging and spitting part....  
2  Okay... then how 'bout we try out some French ...  
3                                         Forget it.  
4                                           Cameron.  


In [2]:
input_sentences = data['speaker1'].tolist()
output_sentences = data['speaker2'].tolist()

In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# 分词函数
def tokenize_sentences(sentences):
    tokenized = [word_tokenize(sentence.lower()) for sentence in sentences]
    return tokenized

input_tokenized = tokenize_sentences(input_sentences)
output_tokenized = tokenize_sentences(output_sentences)


[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_tokenized + output_tokenized)

input_sequences = tokenizer.texts_to_sequences(input_tokenized)
output_sequences = tokenizer.texts_to_sequences(output_tokenized)

vocab_size = len(tokenizer.word_index) + 1


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_tokenized + output_tokenized)

input_sequences = tokenizer.texts_to_sequences(input_tokenized)
output_sequences = tokenizer.texts_to_sequences(output_tokenized)

vocab_size = len(tokenizer.word_index) + 1


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = max(max(len(seq) for seq in input_sequences), max(len(seq) for seq in output_sequences))

input_padded = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
output_padded = pad_sequences(output_sequences, maxlen=max_sequence_length, padding='post')


In [8]:
X = np.array(input_padded)
y = np.array(output_padded)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

embedding_dim = 128
hidden_units = 256

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    LSTM(hidden_units, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 684, 128)          7525504   
_________________________________________________________________
lstm (LSTM)                  (None, 684, 256)          394240    
_________________________________________________________________
dense (Dense)                (None, 684, 58793)        15109801  
Total params: 23,029,545
Trainable params: 23,029,545
Non-trainable params: 0
_________________________________________________________________


In [13]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [15]:
model.fit(X, y, batch_size=16, epochs=5, validation_split=0.2)


Epoch 1/5
  171/11065 [..............................] - ETA: 14:28:49 - loss: 1.0604 - accuracy: 0.9797

KeyboardInterrupt: 